In [6]:
import pandas as pd
import scipy as sp
import numpy as np
import seaborn as sns

In [5]:
for country in ['Egypt', 'Poland', 'Romania']:
    lend_path = 'Total Loans YoY Monthly ' + country + '.csv'
    lend = pd.read_csv(lend_path)
    lend = lend.iloc[26:]
    lend_name = 'Total Loans: YoY: Monthly: ' + country
    lend.rename(columns={'Unnamed: 0': 'Date', lend_name: 'Bank Lending yryr%'}, inplace = True)
    lend['Date'] = pd.to_datetime(lend['Date']) + pd.tseries.offsets.MonthEnd(0)
    lend['Bank Lending yryr%'] = lend['Bank Lending yryr%'].astype(float)

    csv_path = 'Bank_Lending_yryr%_' + country + '.csv'
    lend.to_csv(csv_path, index = False)

In [36]:
lend_path = 'Total Loans YoY Quarterly Hungary.csv'
lend = pd.read_csv(lend_path)
lend = lend.iloc[32:]
lend_name = 'Total Loans: YoY: Quarterly: Hungary'
lend.rename(columns={'Unnamed: 0': 'Date', lend_name: 'Bank Lending Quarterly yryr%'}, inplace = True)
lend['Date'] = pd.to_datetime(lend['Date']) + pd.tseries.offsets.MonthEnd(0)
lend['Bank Lending Quarterly yryr%'] = lend['Bank Lending Quarterly yryr%'].astype(float)
lend = lend[lend.index.values % 3 == 2]
f = sp.interpolate.CubicSpline(lend['Date'], lend['Bank Lending Quarterly yryr%'], bc_type = 'natural')
start = '2007-10'
end = lend['Date'].iloc[-1] + pd.DateOffset(months = 1)
full = pd.date_range(start, end, freq = "M").to_series().reset_index(drop = True)
lendi = pd.Series(f(full))
datai = pd.DataFrame({"Date": full, "Bank Lending yryr%": lendi})

datam = pd.merge(datai, lend, on = 'Date', how = 'left')[['Date', 'Bank Lending yryr%']]
csv_path = 'Bank_Lending_yryr%_Hungary.csv'
datam.to_csv(csv_path, index = False)


In [30]:
for country in ['Egypt', 'Hungary', 'Romania', 'Poland']:
    cpi_path = 'Consumer Price Index YoY Monthly ' + country + '.csv'
    cpi = pd.read_csv(cpi_path)
    cpi = cpi.iloc[26:]
    cpi_name = 'Consumer Price Index: YoY: Monthly: ' + country
    cpi.rename(columns={'Unnamed: 0': 'Date', cpi_name: 'CPI yryrl'}, inplace = True)
    cpi['Date'] = pd.to_datetime(cpi['Date']) + pd.tseries.offsets.MonthEnd(0)
    cpi['CPI yryrl'] = cpi['CPI yryrl'].astype(float)
    cpi['CPI yryrl'] = np.log(cpi['CPI yryrl'] / 100 + 1) * 100

    csv_path = 'CPI_yryrl_' + country + '.csv'
    cpi.to_csv(csv_path, index = False)

In [37]:
for country in ['Egypt', 'Hungary', 'Romania', 'Poland']:
    cpi_path = 'CPI_yryrl_' + country + '.csv'
    cpi = pd.read_csv(cpi_path)
    lend_path = 'Bank_Lending_yryr%_' + country + '.csv'
    lend = pd.read_csv(lend_path)
    data = pd.merge(cpi, lend, on='Date', how='inner')
    data['CPI adjusted Bank Lending'] = data['CPI yryrl'] - data['Bank Lending yryr%']
    csv_path = 'CPI_Bank_Lending_yryr%_' + country + '.csv'
    data.to_csv(csv_path, index = False)